## Simple CNN for road signs

Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

Used classes

In [2]:
USED_CLASSES = [2, 13, 14, 15, 17, 27]
classes_map = [
    "50 speed limit",
    "give way",
    "STOP",
    "no vehicles",
    "no entry",
    "pedestrian crossing"
]

Simple convolutional neural network class

In [3]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 10 * 10, 128),
            nn.ReLU(),
            nn.Linear(128, len(USED_CLASSES))
        )
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

Load and preprocess training data

In [4]:
from torch.utils.data import DataLoader, Subset
from torchvision import transforms, datasets


transform = transforms.Compose([
    transforms.Resize((40, 40)),
    transforms.ToTensor(),
    transforms.Normalize([0.5] * 3, [0.5] * 3)
])

train_dataset = datasets.ImageFolder("images/train", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

Create model, loss function and optimizer

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Training loop

In [6]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch + 1}/{num_epochs}, loss: {round(total_loss, 4)}")

Epoch 1/20, loss: 79.6736
Epoch 2/20, loss: 5.605
Epoch 3/20, loss: 5.3557
Epoch 4/20, loss: 1.3392
Epoch 5/20, loss: 0.3869
Epoch 6/20, loss: 0.9326
Epoch 7/20, loss: 0.2555
Epoch 8/20, loss: 0.1836
Epoch 9/20, loss: 4.042
Epoch 10/20, loss: 0.8307
Epoch 11/20, loss: 0.3691
Epoch 12/20, loss: 0.0728
Epoch 13/20, loss: 0.027
Epoch 14/20, loss: 0.024
Epoch 15/20, loss: 0.0226
Epoch 16/20, loss: 0.0143
Epoch 17/20, loss: 0.0111
Epoch 18/20, loss: 0.0092
Epoch 19/20, loss: 0.0085
Epoch 20/20, loss: 0.0069


Load test data

In [7]:
labels_df = pd.read_csv("images/GT-final_test.csv", sep=";")
labels_df.pop("Roi.X1")
labels_df.pop("Roi.Y1")
labels_df.pop("Roi.X2")
labels_df.pop("Roi.Y2")

class_to_idx = {}
for i in range(len(USED_CLASSES)):
    class_to_idx[USED_CLASSES[i]] = i

labels_df["label"] = labels_df["ClassId"].map(class_to_idx)
labels_df = labels_df.dropna(subset=["label"])
labels_df["label"] = labels_df["label"].astype(int)

Prepare and preprocess test data

In [8]:
from torch.utils.data import Dataset
from PIL import Image
import os


class TestImageDataset(Dataset):
    def __init__(self, df, img_dir, transform = None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row["Filename"])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)
        
        label = row["label"]
        return image, label


test_dataset = TestImageDataset(labels_df, "images/test", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Evaluation loop

In [9]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Total accuracy: {round(accuracy * 100, 2)}%")

Total accuracy: 99.49%


Own image test

In [16]:
img_path = "images/test2.jpg"
img = Image.open(img_path).convert("RGB")
img_tensor = transform(img).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    output = model(img_tensor)
    _, predicted = torch.max(output, 1)

print(f"Prediction: {classes_map[predicted.item()]}")

Prediction: pedestrian crossing


Export model to `pth` file

In [11]:
torch.save(model, "model.pth")